# Set Up

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import gensim
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd

import nltk
from nltk import word_tokenize
nltk.download('punkt')

from scipy.spatial.distance import cosine

pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


# Read in data

## Read in Word2Vec vectors data

In [ ]:
wv = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Colab Notebooks/Octopus2/BlazingText/vectors.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Read in bug data

In [ ]:
dataset = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Octopus2/JIRA_OPEN_DATA_ZOOKEEPER.csv")

In [ ]:
dataset.head()

,id,created,description,key,priority,project,project_name,repositoryname,resolution,resolved,status,title,type,updated,votes,watchers,assignee_id,reporter_id
0,16037,2011-06-16 22:07:44.375,traverseNode in DataTree will never actually t...,ZOOKEEPER-1097,Blocker,ZOOKEEPER,ZooKeeper,ASF,Fixed,2011-06-27 07:30:59.505,Closed,Quota is not correctly rehydrated on snapshot ...,Bug,2011-11-24 03:22:06.217,0,0,7832.0,7832
1,16615,2009-08-07 08:57:50.287,when calling the asynchronous version of creat...,ZOOKEEPER-502,Major,ZOOKEEPER,ZooKeeper,ASF,Fixed,2010-01-27 07:19:07.334,Closed,bookkeeper create calls completion too many times,Bug,2010-03-27 01:24:56.283,0,0,7750.0,7829
2,16189,2010-11-20 06:12:26.124,test -e FILENAME is not support on /bin/sh in ...,ZOOKEEPER-937,Major,ZOOKEEPER,ZooKeeper,ASF,Fixed,2010-12-08 03:02:17.059,Closed,test -e not available on solaris /bin/sh,Bug,2011-11-24 03:21:59.751,0,0,8219.0,8219
3,16616,2009-08-07 06:11:42.25,It timed out according to the console output:h...,ZOOKEEPER-501,Major,ZOOKEEPER,ZooKeeper,ASF,Fixed,2009-08-08 05:03:30.561,Closed,CnxManagerTest failed on hudson,Bug,2009-09-06 06:36:20.729,0,0,7750.0,7750
4,16934,2008-10-07 04:50:15.001,fast leader election test failing .,ZOOKEEPER-178,Major,ZOOKEEPER,ZooKeeper,ASF,Fixed,2008-10-11 07:16:55.237,Closed,FLE test.,Bug,2008-10-26 09:10:43.46,0,0,7750.0,7827


In [ ]:
dataset.shape

(447, 18)

In [ ]:
df = dataset[["id","title","description"]]
df["features"] = df["title"] + " " + df["description"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.iloc[0].description

'traverseNode in DataTree will never actually traverse the limit nodes properly.'

## Pre-proecess the data

In [ ]:
df.features = df.features.str.replace(r"[\n\r\t]+", " ")
df.features = df.features.str.replace(r"([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})", " ")
df.features = df.features.str.replace(r"((http[s]?|ftp):\/)?\/?([^:\/\s]+)((\/\w+)*\/)([\w\-\.]+[^#?\s]+)(.*)?(#[\w\-]+)?", " ")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# Trials

In [ ]:
wv['bug']

array([ 0.36202  ,  0.18874  ,  0.014292 ,  0.018345 , -0.067648 ,
       -0.0038506, -0.20193  ,  0.023967 ,  0.018613 , -0.2605   ,
        0.015813 ,  0.20041  ,  0.10091  , -0.0014307, -0.33163  ,
       -0.15461  ,  0.0092849, -0.31506  ,  0.38153  , -0.020468 ,
        0.33256  , -0.23752  ,  0.40424  ,  0.38221  , -0.20056  ,
       -0.042237 , -0.21792  , -0.057863 , -0.35658  ,  0.42043  ,
        0.10321  ,  0.21758  ,  0.12475  ,  0.13214  , -0.20416  ,
       -0.11204  ,  0.23354  , -0.30087  , -0.49845  , -0.12395  ,
        0.054311 , -0.49731  , -0.069404 ,  0.03734  , -0.113    ,
       -0.32384  ,  0.30196  , -0.30818  ,  0.10555  ,  0.23114  ,
        0.14907  , -0.24687  ,  0.75232  ,  0.023668 , -0.14475  ,
       -0.34755  , -0.091158 ,  0.089999 , -0.11648  , -0.16598  ,
       -0.12745  , -0.32539  , -0.14271  ,  0.22794  ,  0.024229 ,
       -0.05933  ,  0.41251  ,  0.62322  ,  0.067386 ,  0.25472  ,
        0.057332 , -0.097352 ,  0.037767 ,  0.12693  , -0.6950

In [ ]:
print("hello" in wv.vocab)

False


In [ ]:
1 - cosine(wv["bug"], wv["issue"])

0.3196036219596863

In [ ]:
def sentance2vector(sentance):
  tokens = word_tokenize(sentance)
  vector = np.zeros(100)
  for token in tokens:
    if token in wv.vocab:
      vector = vector + wv[token]
  return vector

def cosineSimilarity(v1, v2):
  return 1 - cosine(v1,v2)

In [ ]:
t = 0
feature = df.iloc[t].features
vector = sentance2vector(feature)
vector

array([ 2.93620969,  0.22524796,  1.701629  ,  0.532088  ,  0.22955939,
       -3.98423696, -2.50797306, -0.82110797, -0.89350101, -1.25580599,
       -1.758363  ,  1.7649057 ,  2.26447844, -0.11195838, -3.73000495,
        0.33985903,  2.178708  , -1.39556498,  5.25017007, -4.42579602,
        0.15709999, -2.64368563, -2.43452731,  6.11704623,  3.45482102,
        3.42755905, -1.987559  , -0.0995829 , -1.73860292,  1.445238  ,
       -2.03259093,  4.71634576, -0.29738989,  4.23863598, -0.16602048,
       -4.28064704,  3.88954061, -0.194918  ,  0.24545158, -2.00796898,
        3.27606495, -2.51088699, -3.71881605, -1.42257451,  1.92979769,
        1.395274  ,  2.90137002, -0.97404199, -2.1143351 ,  2.02050015,
       -3.16568191, -4.93915799,  3.59664902, -3.07710712, -4.22112602,
       -1.88674184, -0.50927159,  0.762858  , -1.12717953, -3.40431899,
       -1.17081594, -1.03411102, -4.96640403,  0.29252594,  3.67157991,
       -0.41525806,  0.28964698, -1.29688004, -4.83573991, -0.52

In [ ]:
1 - cosine(vector,vector)

1.0

In [ ]:
df["score"] = np.zeros(len(df))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
for i in range(0,len(df)):
  # print(df.iloc[i].features)
  vect = sentance2vector(df.iloc[i].features)
  df.iloc[i,4] = cosineSimilarity(vector, vect)
  # print(cosineSimilarity(vector, vect))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
df

,id,title,description,features,score
0,16037,Quota is not correctly rehydrated on snapshot ...,traverseNode in DataTree will never actually t...,Quota is not correctly rehydrated on snapshot ...,1.000000
1,16615,bookkeeper create calls completion too many times,when calling the asynchronous version of creat...,bookkeeper create calls completion too many ti...,0.765079
2,16189,test -e not available on solaris /bin/sh,test -e FILENAME is not support on /bin/sh in ...,test -e not available on solaris,0.698046
3,16616,CnxManagerTest failed on hudson,It timed out according to the console output:h...,CnxManagerTest failed on hudson It timed out a...,0.664361
4,16934,FLE test.,fast leader election test failing .,FLE test. fast leader election test failing .,0.626556
...,...,...,...,...,...
442,17104,Stat enchaned to include num of children and size,Moved from SourceForge to Apache.http://source...,Stat enchaned to include num of children and s...,0.721135
443,17105,Use enums rather than ints for types and state,Moved from SourceForge to Apache.http://source...,Use enums rather than ints for types and state...,0.734986
444,17106,List of problem identifiers in zookeeper.h,Submitted on behalf of Jacob. Is is possible f...,List of problem identifiers in zookeeper.h Sub...,0.645257
445,17109,syncLimit has slightly different comments in ...,syncLimit as documented in QuorumPeer is docum...,syncLimit has slightly different comments in ...,0.787938


In [ ]:
df1 = df.loc[df.score > 0.87]

In [ ]:
df2 = df1[["features","score"]]
df2

,features,score
0,Quota is not correctly rehydrated on snapshot reload traverseNode in DataTree will never actually traverse the limit nodes properly.,1.000000
45,"ZooKeeperServer not starting on leader due to a race condition I have found one problem that is causing QuorumPeerMainTest:testQuorum to fail. This test uses 2 ZK servers. The test is failing because leader is not starting ZooKeeperServer after leader election. so everything halts.With the new changes, the server is now started in Leader.processAck() which is called from LeaderHandler. processAck() starts ZooKeeperServer if majority have acked NEWLEADER. The leader puts its ack in the the ackSet in Leader.lead(). Since processAck() is called from LearnerHandler it can happen that the learner's ack is processed before the leader is able to put its ack in the ackSet. When LearnerHandler invokes processAck(), the ackSet for newLeaderProposal will not have quorum (in this case 2). As a result, the ZooKeeperServer is never started on the Leader.The leader needs to ensure that its ack is put in ackSet before starting LearnerCnxAcceptor or invoke processAck() itself after adding to ackSet. I haven't had time to go through the ZAB2 changes so I am not too familiar with the code. Can",0.875181
71,"Zookeeper stop fails if start called twice The zkServer.sh script doesn't check properly to see if a previously startedserver is still running. If you call start twice, the second invocationwill over-write the PID file with a process that then fails due to portoccupancy.This means that stop will subsequently fail.Here is a reference that describes how init scripts should normally work:",0.875309
106,"C Client might not cleanup correctly during close I was looking through the c-client code and noticed a situation where a counter can be incorrectly incremented and a small memory leak can occur.In zookeeper.c : add_completion(), if close_requested is true, then the completion will not be queued. But at the end, outstanding_sync is still incremented and free() never called on the newly allocated completion_list_t. I will submit for review a diff that I believe corrects this issue.",0.879578
171,"explain session expiration better in the docs & faq We are not clear enough (and the diagram we do have seems misleading) on _when_ session expirations are generated. In particular the fact that you only get expirations when the client is connected to the cluster, not when disconnected.we need to detail:1) when do you get expiration2) what is the sequence of events that the watcher sees, from disco state, to getting the expiration (say the expiration happens when the client is disco, what do you see in the watcher while you are getting reconnected)3) we need to give some examples of how to test this. We should be explicit that ""pulling the network cable"" on the client will not show expiration since the cliient will not be reconnected.",0.873472
209,"The last logged zxid calculated by zookeeper servers could cause problems in leader election if data gets corrupted. It is possible that the last loggged zxid as reported by all the servers during leader election is not the last zxid that the server can upload data to. It is very much possible that some transaction or snapshot gets corrupted and the servers actually do not have valid data till last logged zxid. We need to make sure that what the servers report as there last logged zxid, they are able to load data till that zxid.",0.878573
271,"zkCleanup.sh is flaky the zkCleanup.sh script is buggy in two ways:1) it doesn't actually pass through the snapshot count, so it doesn't work2) it assumes that there is only dataDir, it doesn't support dataLogDirAnd it can use cleanup, so that it doesn't blindly call eval from the config file..",0.871896
319,"Issues with FLENewEpochTest I have been able to identify two reasons that cause FLENewEpochTest to fail:1- There is a race condition that is triggered when two peers try to establish a connection to eac